In [29]:
import traceback
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pymysql
from selenium.webdriver.chrome.service import Service
import time

db = pymysql.connect(host="localhost", port=3306, user="root", password="peter2012!", db="sba")
cursor = db.cursor()

options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=/Users/syshin/Desktop/Syshin/SeSAC/파이썬 데이터 처리 프로그래밍/4일차-/MyChrome")
service = Service()
d = webdriver.Chrome(service=service, options=options)
url_melon = "https://www.melon.com/chart/index.htm"
url_instagram = "https://www.instagram.com"

try:
    # 멜론 TOP 100
    d.get(url_melon)
    trs = WebDriverWait(d, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr")))
    already = []
    
    # DB 초기화
    sql_delete_song = 'DELETE FROM SONG;'
    cursor.execute(sql_delete_song)
    sql_delete_singer = 'DELETE FROM SINGER;'
    cursor.execute(sql_delete_singer)
    
    for tr in trs:
        title = tr.find_element(By.CSS_SELECTOR, 'div.rank01 > span > a').text.strip().replace("'", '"')
        singer = tr.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank02 > a').text.strip().replace("'", '"')
        album = tr.find_element(By.CSS_SELECTOR, 'div.ellipsis.rank03 > a').text.strip().replace("'", '"')
        
        # 쿼리 생성 (SINGER)
        if singer not in already:
            sql_singer = f"""
            INSERT INTO SINGER VALUES(NULL, '{singer}', 0)
            """
            cursor.execute(sql_singer)
        already.append(singer) # 중복 확인
        # 쿼리 생성 (SONG)
        sql_song = f"""
        INSERT INTO SONG VALUES(NULL, (SELECT id FROM SINGER WHERE name='{singer}' LIMIT 1), '{title}', '{album}')
        """    
        cursor.execute(sql_song)
    db.commit()
    
    select_sql = "SELECT name from singer"

    cursor.execute(select_sql)
    singer_list = cursor.fetchmany(size=100)
    
    # 인스타 접속, 공식 계정의 팔로워 수 수집, DB에 저장
    d.get(url_instagram)
    authorized = []
    ac = ActionChains(d)
    
    # 검색 버튼
    buttons = WebDriverWait(d, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.xvy4d1p')))
    search_button = buttons[2]
    ac.move_to_element(search_button).click().pause(1).perform()
    first = True
    # 가수마다 공식계정 검색
    for singers in singer_list:
        for singer in singers:
            elem = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input.x7xwk5j")))
            ac.reset_actions()
            ac.send_keys_to_element(elem, singer).pause(2).perform()
            
            
            # 공식계정 있을 시 팔로워수 DB에 저장
            authorized = []
            authorized = d.find_elements(By.CSS_SELECTOR, 'div.x9f619.xjbqb8w.x1rg5ohu.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.xsgj6o6.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1')
            ac.reset_actions()
            if authorized:
                ac.move_to_element(authorized[0]).click().pause(2).perform()
                followers = WebDriverWait(d, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span._ac2a > span')))
                
                # DB 저장
                sql_update_singer = f"""
                    UPDATE singer SET follower = '{followers[1].text}' where name = '{singer}'
                """
                cursor.execute(sql_update_singer)
                
                # 검색 버튼 다시 열기
                if first:
                    buttons = WebDriverWait(d, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.xvy4d1p')))
                    search_button = buttons[2]
                    ac.reset_actions()
                    ac.move_to_element(search_button).click().pause(1).perform()
                    first = False
                else:
                    elem = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input.x7xwk5j")))
                    elem.clear()
                
            # 공식 계정이 없을 시, 검색창 비움
            else:
                time.sleep(1)
                elem = WebDriverWait(d, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input.x7xwk5j")))
                elem.clear()
except Exception as e:
    # print(e)
    traceback.print_exc()
finally:
    d.close()
    d.quit()
    db.commit()
    db.close()

'x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.xdt5ytf.xqjyukv.x1cy8zhl.x1oa3qoh.x1nhvcw1'

In [68]:
'x9f619 xjbqb8w x1rg5ohu x168nmei x13lgxp2 x5pf9jr xo71vjh xsgj6o6 x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1'.replace(" ", ".")

'x9f619.xjbqb8w.x1rg5ohu.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.xsgj6o6.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1'

In [ ]:
#mount_0_0_6B > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div > div > div.x78zum5.xdt5ytf.x1t2pt76.x1n2onr6.x1ja2u2z.x10cihs4 > div.x9f619.xvbhtw8.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.x1q0g3np.xqjyukv.x1qjc9v5.x1oa3qoh.x1qughib > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1.x1dr59a3.xixxii4.x13vifvy.xeq5yr9.x1n327nk > div > div > div.x10l6tqk.x1u3tz30.x1ja2u2z > div > div > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1n2onr6.x1plvlek.xryxfnj.x1iyjqo2.x2lwn1j.xeuugli.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div > div.x6s0dn4.x78zum5.xdt5ytf.x5yr21d.x1odjw0f.x1n2onr6.xh8yej3 > div > a:nth-child(1) > div.x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.xxbr6pl.xbbxn1n.xwib8y2.x1y1aw1k.x1uhb9sk.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1 > div > div > div.x9f619.x1n2onr6.x1ja2u2z.x78zum5.x1iyjqo2.xs83m0k.xeuugli.x1qughib.x6s0dn4.x1a02dak.x1q0g3np.xdl72j9 > div > div > span > span